In [1]:
import os
from glob import glob
from PIL import Image, ImageEnhance, ImageFilter
import numpy as np

# =====================================================
# 1️⃣ DEFINISI PATH DATA
# =====================================================
TEST_DIR = "/content/drive/MyDrive/semester 7/Skripsi/dataset/dataset_ready_custom/test"
ROBUST_DIR = "/content/drive/MyDrive/semester 7/Skripsi/dataset/uji robust"
classes = ["cordana", "healthy", "pestalotiopsis", "sigatoka"]

# Buat folder untuk robust jika belum ada
levels = ["low", "medium", "high"]
transform_types = ["brightness_down", "brightness_up", "blur", "salt_pepper"]

for t in transform_types:
    for l in levels:
        for cls in classes:
            os.makedirs(os.path.join(ROBUST_DIR, t, l, cls), exist_ok=True)

# =====================================================
# 2️⃣ DEFINISI TINGKAT VARIASI
# =====================================================
# Brightness turun: -10%, -20%, -30%
brightness_down_factors = {
    "low": 0.9,
    "medium": 0.8,
    "high": 0.7
}

# Brightness naik: +10%, +20%, +30%
brightness_up_factors = {
    "low": 1.1,
    "medium": 1.2,
    "high": 1.3
}

blur_radius = {
    "low": 1,
    "medium": 2,
    "high": 3
}

# Salt & Pepper density (pengganti noise Gaussian)
sp_density = {
    "low": 0.01,     # 1% pixel rusak
    "medium": 0.03,  # 3%
    "high": 0.06     # 6%
}

# =====================================================
# 4️⃣ FUNGSI SALT & PEPPER
# =====================================================
def add_salt_pepper_noise(img, density):
    arr = np.array(img)
    h, w, c = arr.shape
    n = int(density * h * w)

    # Salt (white)
    ys = np.random.randint(0, h, n)
    xs = np.random.randint(0, w, n)
    arr[ys, xs, :] = 255

    # Pepper (black)
    ys = np.random.randint(0, h, n)
    xs = np.random.randint(0, w, n)
    arr[ys, xs, :] = 0

    return Image.fromarray(arr.astype(np.uint8))

# =====================================================
# 3️⃣ PROSES TRANSFORMASI PER KELAS
# =====================================================
for cls in classes:
    test_paths = glob(os.path.join(TEST_DIR, cls, "*"))

    for img_path in test_paths:
        img_name = os.path.basename(img_path)
        img = Image.open(img_path).convert("RGB")

        # ---------- Brightness Turun ----------
        for level, factor in brightness_down_factors.items():
            img_out = ImageEnhance.Brightness(img).enhance(factor)
            img_out.save(os.path.join(ROBUST_DIR, "brightness_down", level, cls, img_name))

        # ---------- Brightness Naik ----------
        for level, factor in brightness_up_factors.items():
            img_out = ImageEnhance.Brightness(img).enhance(factor)
            img_out.save(os.path.join(ROBUST_DIR, "brightness_up", level, cls, img_name))

        # ---------- Blur ----------
        for level, radius in blur_radius.items():
            img_blur = img.filter(ImageFilter.GaussianBlur(radius=radius))
            img_blur.save(os.path.join(ROBUST_DIR, "blur", level, cls, img_name))

        # ---------- Salt & Pepper ----------
        for level, density in sp_density.items():
            img_sp = add_salt_pepper_noise(img, density)
            img_sp.save(os.path.join(ROBUST_DIR, "salt_pepper", level, cls, img_name))

print("Data test robust berhasil dibuat dengan 3 level untuk brightness, noise, dan blur.")


/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))
/usr/local/lib/python3.12/dist-packages/PIL/JpegImagePlugin.py:887: UserWarning: Image appears to be a malformed MPO file, it will be interpreted as a base JPEG file
  warnings.warn(


Data test robust berhasil dibuat dengan 3 level untuk brightness, noise, dan blur.


In [2]:
import os
import shutil

SRC_DIR = "/content/drive/MyDrive/semester 7/Skripsi/dataset/uji robust"          # <- sumber
DST_DIR = "/content/drive/MyDrive/semester 7/Skripsi/dataset/uji robust fix"      # <- tujuan

transform_types = ["brightness_down", "brightness_up", "blur", "salt_pepper"]
levels = ["low", "medium", "high"]
classes = ["cordana", "healthy", "pestalotiopsis", "sigatoka"]

for t in transform_types:
    for l in levels:
        new_folder = os.path.join(DST_DIR, f"{t}_{l}")
        os.makedirs(new_folder, exist_ok=True)

        for cls in classes:
            src_cls_folder = os.path.join(SRC_DIR, t, l, cls)   # <- ambil dari SRC_DIR
            dst_cls_folder = os.path.join(new_folder, cls)
            os.makedirs(dst_cls_folder, exist_ok=True)

            if not os.path.exists(src_cls_folder):
                print("Tidak ditemukan:", src_cls_folder)
                continue

            for fname in os.listdir(src_cls_folder):
                src_file = os.path.join(src_cls_folder, fname)
                dst_file = os.path.join(dst_cls_folder, fname)

                # Aman: copy dulu (biar tidak hilang kalau error)
                if not os.path.exists(dst_file):
                    shutil.copy2(src_file, dst_file)

print("✅ Struktur folder test robust berhasil dirapikan ke folder fix!")


✅ Struktur folder test robust berhasil dirapikan ke folder fix!


In [3]:
FIX_DIR = "/content/drive/MyDrive/semester 7/Skripsi/dataset/uji robust fix"

print("📊 RINGKASAN JUMLAH GAMBAR DATA ROBUST\n")

for transform in sorted(os.listdir(FIX_DIR)):
    t_path = os.path.join(FIX_DIR, transform)
    if not os.path.isdir(t_path):
        continue

    print(f"📂 {transform}")
    for cls in sorted(os.listdir(t_path)):
        cls_path = os.path.join(t_path, cls)
        if not os.path.isdir(cls_path):
            continue

        num_images = len([
            f for f in os.listdir(cls_path)
            if f.lower().endswith((".jpg", ".jpeg", ".png"))
        ])

        print(f"  ├── {cls:15s}: {num_images} gambar")
    print("-" * 45)

📊 RINGKASAN JUMLAH GAMBAR DATA ROBUST

📂 blur_high
  ├── cordana        : 20 gambar
  ├── healthy        : 20 gambar
  ├── pestalotiopsis : 20 gambar
  ├── sigatoka       : 20 gambar
---------------------------------------------
📂 blur_low
  ├── cordana        : 20 gambar
  ├── healthy        : 20 gambar
  ├── pestalotiopsis : 20 gambar
  ├── sigatoka       : 20 gambar
---------------------------------------------
📂 blur_medium
  ├── cordana        : 20 gambar
  ├── healthy        : 20 gambar
  ├── pestalotiopsis : 20 gambar
  ├── sigatoka       : 20 gambar
---------------------------------------------
📂 brightness_down_high
  ├── cordana        : 20 gambar
  ├── healthy        : 20 gambar
  ├── pestalotiopsis : 20 gambar
  ├── sigatoka       : 20 gambar
---------------------------------------------
📂 brightness_down_low
  ├── cordana        : 20 gambar
  ├── healthy        : 20 gambar
  ├── pestalotiopsis : 20 gambar
  ├── sigatoka       : 20 gambar
-----------------------------------